In [15]:
import wandb
import composuite
from diffusion.utils import *
from diffusion.elucidated_diffusion import Trainer
from diffusion.train_diffuser import SimpleDiffusionGenerator

gin.parse_config_file("/home/anhquanpham/projects/compositional-rl-synth-data/config/diffusion.gin")

base_data_path = '/Users/shubhankar/Developer/compositional-rl-synth-data/data'
base_results_folder = '/Users/shubhankar/Developer/compositional-rl-synth-data/results'

In [17]:
dataset_type = 'expert'

robot = 'IIWA'
obj = 'Hollowbox'
obst = 'None'
task = 'PickPlace'

results_folder = os.path.join(base_results_folder, robot + '_' + obj + '_' + obst + '_' + task)

dataset = load_single_composuite_dataset(base_path=base_data_path, 
                                         dataset_type=dataset_type, 
                                         robot=robot, obj=obj, 
                                         obst=obst, task=task)
dataset = transitions_dataset(dataset)
inputs = make_inputs(dataset)
inputs = torch.from_numpy(inputs).float()
dataset = torch.utils.data.TensorDataset(inputs)

FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = '/Users/shubhankar/Developer/compositional-rl-synth-data/data/expert/IIWA_Hollowbox_None_PickPlace/data.hdf5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)
  In call to configurable 'load_single_composuite_dataset' (<function load_single_composuite_dataset at 0x7f30e5f541f0>)

In [ ]:
diffusion = construct_diffusion_model(inputs=inputs)

In [ ]:
wandb_project = 'offline_rl_diffusion'
wandb_entity = ''
wandb_group = 'diffusion_training'

wandb.init(
    project=wandb_project,
    entity=wandb_entity,
    group=wandb_group,
    name=results_folder.split('/')[-1],
)

In [ ]:
trainer = Trainer(diffusion, dataset, results_folder=results_folder)
trainer.train()

In [ ]:
env = composuite.make(robot, obj, obst, task, use_task_id_obs=True, ignore_done=False)

In [ ]:
generator = SimpleDiffusionGenerator(env=env, ema_model=trainer.ema.ema_model)
observations, actions, rewards, next_observations, terminals = generator.sample(num_samples=100000)

In [ ]:
np.savez_compressed(
    os.path.join(results_folder, 'samples.npz'),
    observations=observations,
    actions=actions,
    rewards=rewards,
    next_observations=next_observations,
    terminals=terminals
)